In [ ]:
pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import transformers
import torchtext
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
import torch.nn as nn
import numpy as np

In [ ]:
#load dataset
dataset = load_dataset("rotten_tomatoes")

#load tokeniser - bert
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#tokenise
def tokenization(example):
    return tokenizer(example["text"], padding="max_length")

#apply function
dataset = dataset.map(tokenization, batched=True)

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [ ]:
#format data for usage in torch
dataset['train'].set_format(type='torch', columns=['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])
dataset['validation'].set_format(type='torch', columns=['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])
dataset['test'].set_format(type='torch', columns=['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
dataset['train']['text'][0]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [ ]:
dataset['train']['input_ids'][0] != 0

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, 

In [ ]:
# PyTorch DataLoader
train_dataloader = DataLoader(dataset['train'], batch_size=8, shuffle=True)
val_dataloader = DataLoader(dataset['validation'], batch_size=8)
test_dataloader = DataLoader(dataset['test'], batch_size=8)

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

# Step 6: Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentimentClassifier(num_classes=2)  # Assuming binary classification (positive/negative sentiment)
model.to(device)

SentimentClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_accuracy = 0
    train_total = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        _, predictedtrain = torch.max(outputs, 1)

        train_total += labels.size(0)
        train_accuracy += (predictedtrain == labels).sum().item()
        train_accuracy_percentage = (train_accuracy / train_total) * 100

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_accuracy = 0
    val_total = 0
    val_losses = []
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask)
            _, predicted = torch.max(outputs, 1)

        valloss = criterion(outputs, labels)
        val_losses.append(valloss.item())

        val_total += labels.size(0)
        val_accuracy += (predicted == labels).sum().item()

    val_accuracy /= val_total
    print(f"Epoch {epoch+1}/{num_epochs}, Train Accuracy: {train_accuracy_percentage:.4f}, Loss:{loss.item()}")
    print(f"Epoch {epoch+1}/{num_epochs}, Val Accuracy: {val_accuracy:.4f}, Loss:{np.mean(val_losses)}")


Epoch 1/10, Train Accuracy: 83.0129, Loss:0.22058479487895966
Epoch 1/10, Val Accuracy: 0.8565, Loss:0.3390183783511617
Epoch 2/10, Train Accuracy: 92.9894, Loss:0.08155964314937592
Epoch 2/10, Val Accuracy: 0.8490, Loss:0.35560007353396667
Epoch 3/10, Train Accuracy: 97.3857, Loss:0.08666671067476273
Epoch 3/10, Val Accuracy: 0.8649, Loss:0.4157205445365285
Epoch 4/10, Train Accuracy: 98.7339, Loss:0.0011845396365970373
Epoch 4/10, Val Accuracy: 0.8452, Loss:0.66538157256354
Epoch 5/10, Train Accuracy: 98.8277, Loss:0.0012415691744536161
Epoch 5/10, Val Accuracy: 0.8546, Loss:0.6395529172141731
Epoch 6/10, Train Accuracy: 99.2380, Loss:0.00013630582543555647
Epoch 6/10, Val Accuracy: 0.8527, Loss:0.7000723634762346
Epoch 7/10, Train Accuracy: 99.1559, Loss:0.0010046218521893024
Epoch 7/10, Val Accuracy: 0.8518, Loss:0.6059612197564296
Epoch 8/10, Train Accuracy: 99.3787, Loss:0.0002643601910676807
Epoch 8/10, Val Accuracy: 0.8508, Loss:0.8429618004536054
Epoch 9/10, Train Accuracy: 99

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(model.state_dict(), 'model10.pth')

Mounted at /content/drive


In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/model10.pth')


In [ ]:
torch.save(model.state_dict(), 'mo.pth')

In [ ]:
model

SentimentClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
torch.max(model(dataset['train']['input_ids'][10000:10005].to(device),torch.zeros(dataset['train']['input_ids'][10000:10005].shape).to(device)),1)

torch.return_types.max(
values=tensor([], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([], device='cuda:0', dtype=torch.int64))

Predict on test set

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize lists to store predictions and labels
all_predictions = []
all_labels = []

model.eval()

# Iterate over batches in the test dataloader
for batch in test_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, predicted = torch.max(outputs, 1)

    # Collect predictions and labels
    all_predictions.extend(predicted.cpu().tolist())
    all_labels.extend(labels.cpu().tolist())

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)

print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1-score:", f1)

Test Accuracy: 0.8480300187617261
Test Precision: 0.8702594810379242
Test Recall: 0.8180112570356473
Test F1-score: 0.8433268858800774
